<a href="https://colab.research.google.com/github/emerson-matos/tcc-notebooks/blob/main/sentimentos/bert-playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import pandas as pd
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
date = '2023-09-01'
filename = 'analisar.csv'
dados = pd.read_csv(f"/content/gdrive/MyDrive/tcc/exploratoria/{date}-{filename}")

dados.head()[['text', 'source', 'data_avaliacao']]

In [ ]:
treinado = pd.read_csv(f"/content/gdrive/MyDrive/tcc/testando-classificacao.csv")

treinado.head()[['text', 'source', 'data_avaliacao']]

In [ ]:
dados.describe()

,rating,likes,mes_avaliacao,ano_avaliacao
count,49219.000000,49219.000000,49219.000000,49219.000000
mean,4.604787,0.166623,6.790650,2020.699181
std,0.895289,0.731727,1.524495,1.746374
min,1.000000,0.000000,1.000000,2018.000000
25%,5.000000,0.000000,7.000000,2019.000000
50%,5.000000,0.000000,7.000000,2021.000000
75%,5.000000,0.000000,7.000000,2022.000000
max,5.000000,25.000000,12.000000,2023.000000


In [6]:
!pip install transformers[sentencepiece] datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00


# bert

https://medium.com/data-hackers/an%C3%A1lise-de-sentimentos-em-portugu%C3%AAs-utilizando-pytorch-e-python-91a232165ec0

https://sol.sbc.org.br/index.php/stil/article/view/17784

https://towardsdatascience.com/sentiment-analysis-in-10-minutes-with-bert-and-hugging-face-294e8a04b671

In [7]:
sentences = [
  'Foi muito bom, exceto o primeiro atendimento na recepção na chegada, fizemos o chek-in e não nos informaram onde ficaria o nosso chalé, tivemos que ficar perguntando a alguns funcionários, fora isso tudo de bom.',
  'Lindo d mais',
  'Muito bom. Vale o valor pago',
  'Tudo perfeito.',
  'Drinks muito bom',
  'Incrível',
  'Foi ótimo dias maravilhosos...',
  'Passagem incrível.  Boa comida.  Piscinas amplas com águas mornas.',
  'Alimentação péssima! Do mais tudo ok!',
  'Muito sofisticado',
  'Otima',
  'Lugar fantástico, recomendo super.',
  'Estou deslumbrado com a beleza ,ao entorno do hotel ,quarto amplo e confortável ,a piscina espetacular ,foi uma experiência incrível!!!'
]

In [8]:
import torch
import transformers
from transformers import BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup, pipeline, AutoTokenizer, AutoModelForSequenceClassification

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

teste = dados.copy()

teste.head(5)

NameError: ignored

In [10]:
PRE_TRAINED_MODEL_NAME = 'philschmid/distilbert-base-multilingual-cased-sentiment'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = AutoTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(PRE_TRAINED_MODEL_NAME).to(device)

In [11]:
nlp = pipeline('sentiment-analysis',
               model=model,
               tokenizer=tokenizer,
               max_length=512,
               truncation=True,
               padding='max_length',
               pad_to_max_length=True,
               device=device,
               framework='pt',
               batch_size=16,
)

In [12]:
nlp("Meu cachorro é muito top")

[{'label': 'positive', 'score': 0.9314833283424377}]

In [13]:
nlp(sentences)

[{'label': 'negative', 'score': 0.9226688146591187},
 {'label': 'neutral', 'score': 0.4194216728210449},
 {'label': 'negative', 'score': 0.8745063543319702},
 {'label': 'positive', 'score': 0.9755363464355469},
 {'label': 'negative', 'score': 0.7377951741218567},
 {'label': 'negative', 'score': 0.5013457536697388},
 {'label': 'negative', 'score': 0.48443007469177246},
 {'label': 'neutral', 'score': 0.5337892770767212},
 {'label': 'negative', 'score': 0.9250680804252625},
 {'label': 'neutral', 'score': 0.4281403720378876},
 {'label': 'positive', 'score': 0.8648583292961121},
 {'label': 'positive', 'score': 0.9849022030830383},
 {'label': 'positive', 'score': 0.9673092365264893}]

In [14]:
classifications = [model.config.id2label[output] for output in range(len(model.config.id2label))]
print(classifications)

['negative', 'neutral', 'positive']


In [16]:
input_ids = tokenizer("Olá, meu cachorro é uma gracinha", return_tensors="pt", add_special_tokens=True).to(device)  # Batch size 1
outputs = model(**input_ids).logits

classifications = torch.argmax(outputs, axis=1)
print(classifications)

tensor([2])


In [17]:
classifications.item()

2

In [18]:
classifications = model.config.id2label[classifications.item()]
print(classifications)

positive


In [19]:
outputs

tensor([[-0.3225,  0.1047,  0.2098]], grad_fn=<AddmmBackward0>)

In [20]:
class_names = classifications

In [ ]:
%mkdir -p gdrive/MyDrive/tcc/bert
file_csv_data = 'gdrive/MyDrive/tcc/bert/reviews.csv'
df.to_csv(file_csv_data, index=None, header=True)

In [ ]:
sample_txt = df.loc[1]['text']
sample_txt

In [26]:
tokens = tokenizer.tokenize(sentences[0])
token_ids = tokenizer.convert_tokens_to_ids(tokens)

print(f' Sentence: {sentences[0]}')
print(f'   Tokens: {tokens}')
print(f'Token IDs: {token_ids}')

 Sentence: Foi muito bom, exceto o primeiro atendimento na recepção na chegada, fizemos o chek-in e não nos informaram onde ficaria o nosso chalé, tivemos que ficar perguntando a alguns funcionários, fora isso tudo de bom.
   Tokens: ['Foi', 'muito', 'bom', ',', 'ex', '##ceto', 'o', 'primeiro', 'at', '##endi', '##mento', 'na', 're', '##cepção', 'na', 'chegada', ',', 'fi', '##zem', '##os', 'o', 'che', '##k', '-', 'in', 'e', 'não', 'nos', 'informa', '##ram', 'onde', 'ficar', '##ia', 'o', 'nos', '##so', 'cha', '##lé', ',', 'ti', '##vem', '##os', 'que', 'ficar', 'per', '##gun', '##tando', 'a', 'alguns', 'fun', '##cionários', ',', 'fora', 'isso', 'tudo', 'de', 'bom', '.']
Token IDs: [13727, 16159, 32965, 117, 11419, 95133, 183, 13473, 10160, 77089, 11498, 10132, 11639, 79574, 10132, 70657, 117, 14045, 25464, 10310, 183, 10262, 10174, 118, 10106, 173, 11420, 11573, 93039, 13845, 12058, 86070, 10280, 183, 11573, 11669, 18939, 15699, 117, 14382, 42201, 10310, 10121, 86070, 10178, 32657, 32338,

In [27]:
encoding = tokenizer.encode_plus(
  sentences[0],
  max_length=45,
  add_special_tokens=True, # Add '[CLS]' and '[SEP]'
  return_token_type_ids=False,
  pad_to_max_length=True,
  return_attention_mask=True,
  return_tensors='pt',  # Return PyTorch tensors
)

encoding.keys()

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


dict_keys(['input_ids', 'attention_mask'])

In [28]:
print(len(encoding['input_ids'][0]))
encoding['input_ids'][0]

45


tensor([  101, 13727, 16159, 32965,   117, 11419, 95133,   183, 13473, 10160,
        77089, 11498, 10132, 11639, 79574, 10132, 70657,   117, 14045, 25464,
        10310,   183, 10262, 10174,   118, 10106,   173, 11420, 11573, 93039,
        13845, 12058, 86070, 10280,   183, 11573, 11669, 18939, 15699,   117,
        14382, 42201, 10310, 10121,   102])

In [29]:
print(tokenizer.convert_ids_to_tokens(encoding['input_ids'][0]))

['[CLS]', 'Foi', 'muito', 'bom', ',', 'ex', '##ceto', 'o', 'primeiro', 'at', '##endi', '##mento', 'na', 're', '##cepção', 'na', 'chegada', ',', 'fi', '##zem', '##os', 'o', 'che', '##k', '-', 'in', 'e', 'não', 'nos', 'informa', '##ram', 'onde', 'ficar', '##ia', 'o', 'nos', '##so', 'cha', '##lé', ',', 'ti', '##vem', '##os', 'que', '[SEP]']


In [ ]:
token_lens = []

for txt in df.text:
  tokens = tokenizer.encode(txt, max_length=512)
  token_lens.append(len(tokens))

In [ ]:
# deprecated sns.distplot(token_lens)
sns.histplot(token_lens,
             kde=True,
             stat="density",
             kde_kws=dict(cut=3),
             alpha=.4,
             edgecolor=(1, 1, 1, .4),
             )
plt.xlim([0, 600]);
plt.xlabel('Token count');

In [ ]:
max(token_lens)

In [ ]:
MAX_LEN = 512

In [30]:
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

device

device(type='cpu')

In [32]:
class GPReviewDataset(Dataset):

  def __init__(self, reviews, targets, tokenizer, max_len):
    self.reviews = reviews
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.reviews)

  def __getitem__(self, item):
    review = str(self.reviews[item])
    target = self.targets[item]

    encoding = self.tokenizer.encode_plus(
      review,
      add_special_tokens=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      #padding='longest',
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt',
    )

    return {
      'review_text': review,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'targets': torch.tensor(target, dtype=torch.long)
    }

In [33]:
df_train, df_test = train_test_split(df, test_size=0.1, random_state=RANDOM_SEED)
df_val, df_test = train_test_split(df_test, test_size=0.5, random_state=RANDOM_SEED)

df_train.shape, df_val.shape, df_test.shape

NameError: ignored

In [ ]:
def create_data_loader(df, tokenizer, max_len, batch_size):
  ds = GPReviewDataset(
    reviews=df.text.to_numpy(),
    targets=df.sentiment.to_numpy(),
    tokenizer=tokenizer,
    max_len=max_len
  )

  return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers=4
  )

In [ ]:
BATCH_SIZE = 16

train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)

In [ ]:
len(train_data_loader)

In [ ]:
data = next(iter(train_data_loader))
data.keys()

In [ ]:
print(data['input_ids'].shape)
print(data['attention_mask'].shape)
print(data['targets'].shape)

In [ ]:
class SentimentClassifier(torch.nn.Module):

  def __init__(self, n_classes):
    super(SentimentClassifier, self).__init__()
    self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME, return_dict=False)
    self.drop = torch.nn.Dropout(p=0.3)
    #The last_hidden_state is a sequence of hidden states of the last layer of the model
    self.out = torch.nn.Linear(self.bert.config.hidden_size, n_classes)

  def forward(self, input_ids, attention_mask):
    _, pooled_output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask
    )
    output = self.drop(pooled_output)
    return self.out(output)


In [ ]:
model = SentimentClassifier(len(class_names))
model = model.to(device)

In [ ]:
EPOCHS = 10

optimizer = AdamW(model.parameters(), lr=3e-5, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)

loss_fn = torch.nn.CrossEntropyLoss().to(device)

In [ ]:
def train_epoch(
  model,
  data_loader,
  loss_fn,
  optimizer,
  device,
  scheduler,
  n_examples
):
  model = model.train()

  losses = []
  correct_predictions = 0

  for d in data_loader:
    input_ids = d["input_ids"].to(device)
    attention_mask = d["attention_mask"].to(device)
    targets = d["targets"].to(device)

    outputs = model(
      input_ids=input_ids,
      attention_mask=attention_mask
    )

    _, preds = torch.max(outputs, dim=1)
    loss = loss_fn(outputs, targets)

    correct_predictions += torch.sum(preds == targets)
    losses.append(loss.item())

    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()

  return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
def eval_model(model, data_loader, loss_fn, device, n_examples):
  model = model.eval()

  losses = []
  correct_predictions = 0

  with torch.no_grad():
    for d in data_loader:
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)

      loss = loss_fn(outputs, targets)

      correct_predictions += torch.sum(preds == targets)
      losses.append(loss.item())

  return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
%%time
from collections import defaultdict
history = defaultdict(list)
best_accuracy = 0

for epoch in range(EPOCHS):

  print(f'Epoch {epoch + 1}/{EPOCHS}')
  print('-' * 10)

  train_acc, train_loss = train_epoch(
    model,
    train_data_loader,
    loss_fn,
    optimizer,
    device,
    scheduler,
    len(df_train)
  )

  print(f'Train loss {train_loss} accuracy {train_acc}')

  val_acc, val_loss = eval_model(
    model,
    val_data_loader,
    loss_fn,
    device,
    len(df_val)
  )

  print(f'Val   loss {val_loss} accuracy {val_acc}')
  print()

  history['train_acc'].append(train_acc)
  history['train_loss'].append(train_loss)
  history['val_acc'].append(val_acc)
  history['val_loss'].append(val_loss)

  if val_acc > best_accuracy:
    torch.save(model.state_dict(), 'best_model_state.bin')
    best_accuracy = val_acc

In [ ]:
history

In [ ]:
history['train_acc']

In [ ]:
plt.plot(history['train_acc'], label='train accuracy')
plt.plot(history['val_acc'], label='validation accuracy')

plt.title('Training history')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.ylim([0, 1]);

In [ ]:
def get_predictions(model, data_loader):
  model = model.eval()

  review_texts = []
  predictions = []
  prediction_probs = []
  real_values = []

  with torch.no_grad():
    for d in data_loader:

      texts = d["review_text"]
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)

      probs = F.softmax(outputs, dim=1)

      review_texts.extend(texts)
      predictions.extend(preds)
      prediction_probs.extend(probs)
      real_values.extend(targets)

  predictions = torch.stack(predictions).cpu()
  prediction_probs = torch.stack(prediction_probs).cpu()
  real_values = torch.stack(real_values).cpu()
  return review_texts, predictions, prediction_probs, real_values

In [ ]:
y_review_texts, y_pred, y_pred_probs, y_test = get_predictions(
  model,
  test_data_loader
)

In [ ]:
print(classification_report(y_test, y_pred, target_names=class_names))

In [ ]:
def show_confusion_matrix(confusion_matrix):
  hmap = sns.heatmap(confusion_matrix, annot=True, fmt="d", cmap="Blues")
  hmap.yaxis.set_ticklabels(hmap.yaxis.get_ticklabels(), rotation=0, ha='right')
  hmap.xaxis.set_ticklabels(hmap.xaxis.get_ticklabels(), rotation=30, ha='right')
  plt.ylabel('True sentiment')
  plt.xlabel('Predicted sentiment');

cm = confusion_matrix(y_test, y_pred)
df_cm = pd.DataFrame(cm, index=class_names, columns=class_names)
show_confusion_matrix(df_cm)